## Two-dimensional projectile motion with quadratic drag

In this notebook, I will be considering the numerical solution to the equations of motion concerning the 2D trajectory of a projectile moving through a medium that exerts a quadratic drag force on it. For such a situation, we can express Newton's laws of motion as: 

\begin{align*}
\mathbf{F} = m\ddot{\mathbf{x}} = m \dot{\mathbf{v}} = 
\begin{bmatrix}
-\beta v v_x \\
-mg -\beta v v_y
\end{bmatrix}
\end{align*}

where $v = v(v_x, v_y) = \sqrt{v_x^2+v_y^2}$, $\beta$ is the quadratic drag coefficient, $g$ is the gravitational constant, and $m$ is the mass of the system. This is a first order coupled differential equation, with $\dot{v}_x = f(v_x, v_y, t)$ and $\dot{v}_y = g(v_x, v_y, t)$.

To solve this, we can define the state function $\mathbf{f}$


\begin{align*}
\mathbf{f}(v_x, v_y, t) = 
\begin{bmatrix}
f(v_x, v_y, t) \\
g(v_x, v_y, t)
\end{bmatrix}
=
\begin{bmatrix}
-\frac{\beta}{m} \sqrt{v_x^2+v_y^2} v_x \\
-g -\frac{\beta}{m} \sqrt{v_x^2+v_y^2} v_y
\end{bmatrix}
\end{align*}

This function can then be fed into our scipy.integrate.ivp_solve function in order to solve for the velocity.

Finally, to get the position $\mathbf{x}(t)$, we can integrate our velocity:

\begin{align*}
\mathbf{x}(t) = \int_{t_0}^t \mathbf{v}(t') dt' \simeq 
\sum_{i = 1}^N \mathbf{v}_i \Delta t_i
\end{align*}

In the last step, we express the computed velocities as $\mathbf{v}_i = \begin{bmatrix} v_{xi} \\ v_yi \end{bmatrix}$ determined at each time-step $\Delta t_i$ from our Runge Kutta method. This integral can be computed using a cumulative trapezoidal or cumulative Simpsom algorithm from scipy.integrate.

In [1]:
import numpy as np
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import ipywidgets as wd

In [ ]:
def stateFunction(t, vstar, beta, g, m):
    vx, vy = vstar
    vxDot = -(beta/m)*np.sqrt(vx**2 + vy**2)*vx
    vyDot = -g - (beta/m)*np.sqrt(vx**2 + vy**2)*vy
    return [vxDot, vyDot]

def integrateVelocity(t, vel, r0):
    x0, y0 = r0
    vx, vy = vel
    xOfT = x0 + sc.integrate.